In [ ]:
def preprocessing(target):
    target = re.sub(r'\n(?!\|)', ', ', target)
    return target.replace('*','').replace('[', '').replace(']', '')

def getAttribute(attribute, target):
    target = preprocessing(target)
    matchingString = f'\|{attribute}=([^\n\{{\}}<>]*)[^\n]*\n\|'
    result="?"
    if attribute == "Mort" or attribute == "Naissance":
        try:
            result = re.search('\|{}=.*?(\d{4}).*?\s\|',target).group(1)
        except:
            return "0"
    else:
        #if attribute == "Sang":
            #matchingString = "\|Sang=\[\[([^]]+)\]\]"
        #if attribute == "Localisation":
            #matchingString = "\|Localisation=.*?\[\[([^]]+)\]\]"
        #if attribute == "Statut":
            #matchingString = "\|Statut=([^\{\}\n]*).*\n\|"
        try:
            result = re.search(matchingString, target).group(1)
            if result == "":
                result="?"
        except:
            return "?"
    result = result.replace("'","('')")
    return result

def getNom(target):
    try:
        result = re.search('\|Nom=(.*?)\s\|', target).group(1)
    except:
        return ""
    return result

def getYeux(target):
    try:
        result = re.search('\|Yeux=([\w\ 0-9,\']*).*?\s\|', target).group(1)
    except:
        return ""
    return result

def getGenre(target):
    try:
        result = re.search('\|Genre=\ ?([A-Za-z0-9â]*?)\s\|', target).group(1)
    except:
        return ""
    return result

def getCheveux(target):
    try:
        result = re.search('\|Cheveux=([A-Za-z0-9]*?)\s\|', target).group(1)
        if result == "":
            return "Inconnu"
    except:
        return "Inconnu"
    return result


def getNaissance(target):
    try:
        result = re.search('\|Naissance=.*?(\d{4}).*?\s\|',target).group(1)
    except:
        return "0"
    return result

def getMort(target):
    try:
        result = re.search('\|Mort=.*?(\d{4}).*?\s\|',target).group(1)
    except:
        return "0"
    return result


# with open('personnagesnaissance.txt','x') as f:
#     for page in pages:
#         if "[[Catégorie:Personnage]]" in page.text and ("Catégorie:" not in page.title.text):
#             f.write(getAttribute("Nom",page.text) +" "+ getAttribute("Naissance",page.text)+"\n")
for page in pages[0:100]:
    if "[[Catégorie:Personnage]]" in page.text and ("Catégorie:" not in page.title.text):
        id = ""
        for attr in ["Nom","Cheveux","Yeux","Genre","Patronus","Sang"]:
            id += f" {getAttribute(attr,page)}"
        id += str(getMort(page.text))
        print(id)
for page in pages[100:2000]:
    if "[[Catégorie:Créatures" in page.text and ("Catégorie:" not in page.title.text and "Portail:" not in page.title.text):
        nom = getAttribute("Nom",page)
        yeux = getAttribute("Yeux",page)
        statut = getAttribute("Statut",page)
        apparence = getAttribute("Apparence",page)
        caractère = getAttribute("Caractère",page)
        localisation = getAttribute("Localisation",page)
        print(f"Insert INTO créatures VALUES('{nom}','{statut}','{yeux}','{apparence}','{caractère}','{localisation}')")

In [ ]:
with open('personnages.txt','x') as f:
    for page in pages:
        if "[[Catégorie:Personnage]]" in page.text and ("Catégorie:" not in page.title.text):
            f.write(xmlToSql.getAttribute("Nom",page) +" "+ xmlToSql.getAttribute("Yeux",page)+"\n")

In [1]:
from bs4 import BeautifulSoup
import re
from xmlToSql import getAttribute
import sqlite3

Reading and parsing the dump with BeautifulSoup

In [2]:
with open('frharrypotter_pages_current.xml','r') as f:
    file = f.read()

soup = BeautifulSoup(file, 'xml')
pages = soup.find_all('page')

## Creating the databases

Start by creating an empty database.

In [4]:
connection = sqlite3.connect("harrypotter_fr.db")
cursor = connection.cursor()

### First the character table

Create the table for characters named "personnages".

Fill in the records thanks to the dump of the french Harry Potter Fandom and parsing functions in the xmlToSql.py file.

In [5]:
cursor.execute("CREATE TABLE personnages (nom TEXT NOT NULL PRIMARY KEY, genre TEXT, yeux TEXT, cheveux TEXT, naissance INTEGER, mort INTEGER, patronus TEXT, sang TEXT)")
for page in pages:
    if "[[Catégorie:Personnage]]" in page.text and ("Catégorie:" not in page.title.text):
        nom = getAttribute("Nom",page)
        yeux = getAttribute("Yeux",page)
        cheveux = getAttribute("Cheveux",page)
        genre = getAttribute("Genre",page)
        patronus = getAttribute("Patronus",page)
        naissance = getAttribute("Naissance",page)
        mort = getAttribute("Mort",page)
        sang = getAttribute("Sang",page)
        # print(f"Insert INTO personnages VALUES('{nom}','{genre}','{yeux}','{cheveux}','{naissance}','{mort}','{patronus}','{sang}')")
        try:
            if nom =='?':
                continue
            cursor.execute(f"Insert INTO personnages VALUES('{nom}','{genre}','{yeux}','{cheveux}','{naissance}','{mort}','{patronus}','{sang}')")
        except:
            print(f"Already inserted {nom}")

Already inserted Lily Potter
Already inserted Fred Weasley
Already inserted Molly Weasley
Already inserted Avery
Already inserted Mulciber
Already inserted Élève garçon de Beauxbâtons non identifié
Already inserted Élève garçon de Beauxbâtons non identifié
Already inserted Élèves filles de Beauxbâtons non identifiées
Already inserted Stebbins
Already inserted ?
Already inserted Emily
Already inserted Derek
Already inserted ?
Already inserted M. Carneirus
Already inserted Stebbins
Already inserted Julia Dehoff
Already inserted M. Carneiro
Already inserted Emma Vane
Already inserted Luca Caruso
Already inserted Albert
Already inserted Andrew Snowy Owl
Already inserted Winky Crockett
Already inserted Jo King
Already inserted Emma Vanity
Already inserted L. Wakefield
Already inserted William de Cognac
Already inserted Eduardo Lima
Already inserted Eduardus Limus
Already inserted Miraforum
Already inserted Loretta Fieldwake
Already inserted M. Mina
Already inserted R. Amorim


### Then table for creatures

In [6]:
cursor.execute("CREATE TABLE créatures (nom_créature TEXT NOT NULL PRIMARY KEY, statut_créature TEXT, yeux_créature TEXT, plumes_poils text, apparence TEXT, caractère TEXT, localisation TEXT)")

In [8]:
for page in pages:
    if "[[Catégorie:Créature" in page.text and ("Catégorie:" not in page.title.text and "Portail:" not in page.title.text):
        nom = getAttribute("Nom",page)
        yeux = getAttribute("Yeux",page)
        poils = getAttribute("Plumes/Poils",page)
        statut = getAttribute("Statut",page)
        apparence = getAttribute("Apparence",page)
        caractère = getAttribute("Caractère",page)
        localisation = getAttribute("Localisation",page)
        #print(f"Insert INTO créatures VALUES('{nom}','{statut}','{yeux}','{apparence}','{caractère}','{localisation}')")
        try:
            cursor.execute(f"Insert INTO créatures VALUES('{nom}','{statut}','{yeux}','{poils}','{apparence}','{caractère}','{localisation}')")
        except:
            print(f"Already inserted {nom}")

Already inserted Diablotin
Already inserted ?
Already inserted ?
Already inserted Cachalot
Already inserted Hippocampe


## Finally Junction Table between family members

In [10]:
cursor.execute("CREATE TABLE famille (premier_nom TEXT NOT NULL, second_nom TEXT NOT NULL, relation TEXT, FOREIGN KEY (premier_nom) REFERENCES personnages(nom), FOREIGN KEY (second_nom) REFERENCES personnages(nom))")

In [11]:
def insertRelative(first,second,relation):
    first = first.replace("'","('')")
    second = second.replace("'","('')")
    relation = relation.replace("'","('')")
    try:
        cursor.execute(f"Insert INTO famille VALUES('{first}','{second}','{relation}')")
    except:
        print(f"Inserting {first} {second} {relation} didn't work")
    return

pattern = re.compile(r'Famille\s*=\s*(.*?)\s\|', re.S)
for page in pages:
    if "[[Catégorie:Personnage]]" in page.text and ("Catégorie:" not in page.title.text and "Utilisateur:" not in page.title.text):
        nom = getAttribute("Nom",page)
        if re.search('Famille\s*=\s*\n\|',page.text): #Related to no-one
            continue
        singleRelative = re.search('Famille\s*=\s*\[\[(.*?)\]\]',page.text)
        if singleRelative:#Related to one person
            onlyRelative = singleRelative.group(1)
            if "|" in onlyRelative:
                parts = onlyRelative.split("|")
                insertRelative(parts[0],nom,parts[1])
            else:
                relation = re.search('Famille=.*?{{Comment\|(.*?)}}',page.text)
                if relation:
                    insertRelative(onlyRelative,nom,relation.group(1))
                else:
                    insertRelative(onlyRelative,nom,"?")
            continue
        multipleRelatives = pattern.search(page.text)
        if multipleRelatives:
            famille_text = multipleRelatives.group(1)
            famille = re.findall(r'\*\[\[([^\]]+)\]\]',famille_text)
            for member in famille:
                if "|" in member:
                    parts = member.split("|")
                    insertRelative(parts[0],nom,parts[1])
                else:
                    relation = re.search(f'\*\[\[{member}\]\].*?{{{{Comment\|(.*?)}}}}',page.text)
                    if relation:
                        insertRelative(member,nom, relation.group(1))
            
        

Finally commit the changes to the database so that all changes are saved and close the connection

In [12]:
connection.commit()
cursor.close()
connection.close()

# Jetzt auf Deutsch

In [1]:
from bs4 import BeautifulSoup
import re
from xmlToSql_de import getAttribute, randomly_attribute
import sqlite3
import random

In [2]:
with open('deharrypotter_pages_current.xml','r') as f:
    file = f.read()

soup = BeautifulSoup(file, 'xml')
pages = soup.find_all('page')

In [3]:
connection = sqlite3.connect("harrypotter_de.db")
cursor = connection.cursor()

In [4]:
cursor.execute("CREATE TABLE charaktere (Name TEXT NOT NULL PRIMARY KEY, Geschlecht TEXT, Augen TEXT, Haare TEXT, Geburt INTEGER, Tod INTEGER, Patronus TEXT, Blut TEXT)")
kategorien =["[[Kategorie:Weibliche Charaktere]]", "Kategorie:Hexen / Zauberer", "Kategorie:Männliche Charaktere"]
for page in pages:
    if any(kategorie in page.text for kategorie in kategorien) and ("Kategorie:" not in page.title.text):
        if not "Charakterinfobox" in page.text and not "Infobox" in page.text:
            nom = page.title.text
            yeux = randomly_attribute("Augen")
            cheveux = randomly_attribute("Haare")
            if "Der" in nom:
                genre = "Männlich"
            elif "Die" in nom:
                genre = "Weiblich"
            else:
                genre = random.choices(["Männlich","Weiblich"])[0]
            naissance = 0
            mort = 0
            sang = "?"
            patronus = "?"
        else:
            nom = getAttribute("Name",page)
            yeux = getAttribute("Augen",page)
            cheveux = getAttribute("Haare",page)
            genre = getAttribute("Geschlecht",page)
            patronus = getAttribute("Patronus",page)
            naissance = getAttribute("Geburt",page)
            mort = getAttribute("Tod",page)
            sang = getAttribute("Blut",page)
        if (nom == "?"):
            print(page.title.text)
        #print(f"Insert INTO personnages VALUES('{nom}','{genre}','{yeux}','{cheveux}','{naissance}','{mort}','{patronus}','{sang}')")
        #cursor.execute(f"Insert INTO charaktere VALUES('{nom}','{genre}','{yeux}','{cheveux}','{naissance}','{mort}','{patronus}','{sang}')")
        try:
            cursor.execute(f"Insert INTO charaktere VALUES('{nom}','{genre}','{yeux}','{cheveux}','{naissance}','{mort}','{patronus}','{sang}')")
        except:
            print(f"Already inserted {nom}")

Walburga Black
Alphard Black
Already inserted ?
Orion Black
Already inserted ?
Alastor Moody
Already inserted ?
Alecto Carrow
Already inserted ?
Already inserted Sirius Black
Astoria Malfoy
Already inserted ?
Bathilda Bagshot
Already inserted ?
Bertha Jorkins
Already inserted ?
Marlene McKinnon
Already inserted ?
Edgar Bones
Already inserted ?
Mundungus Fletcher
Already inserted ?
Vorlage:Charakterinfobox Zauberer
Already inserted ?
Amycus Carrow
Already inserted ?
Armando Dippet
Already inserted ?
Ignotus Peverell
Already inserted ?
Augustus Rookwood
Already inserted ?
Rabastan Lestrange
Already inserted ?
Albert Runcorn
Already inserted ?
Amelia Bones
Already inserted ?
Laura Madley
Already inserted ?
Andrew Kirke
Already inserted ?
Already inserted Cygnus Black
Andromeda Tonks
Already inserted ?
Vater von Dean Thomas
Already inserted ?
Already inserted Sirius Black
Already inserted Arcturus Black
Already inserted Cygnus Black
Already inserted Arcturus Black
Already inserted Riese in

In [6]:
print(cursor.execute(f"SELECT COUNT(*) from charaktere").fetchall())

[(801,)]


## Kreaturen

In [7]:
cursor.execute("CREATE TABLE kreaturen (name_kreatur TEXT NOT NULL PRIMARY KEY, status_kreatur TEXT, augen_kreatur TEXT, haut_kreatur text, federn_kreatur, eigenschaften_kreatur TEXT, verbreitung_kreatur TEXT)")

In [8]:
kategorien = ["Kategorie:Tierwesen","Kategorie:Vögel","Kategorie:Kreatur","Kategorie:XXX Kreaturen","Kategorie:Flugfähige Kreaturen","Kategorie:Verstorbene Kreaturen","Kategorie:Sprechende Kreaturen","Kategorie:Magische Kreaturen"]
for page in pages:
    if any(kategorie in page.text for kategorie in kategorien) and ("Kategorie:" not in page.title.text and "Portal:" not in page.title.text and "Vorlage:" not in page.title.text):
        if not "Charakterinfobox" in page.text and not "Infobox" in page.text:
            continue
        else:
            nom = getAttribute("Name",page)
            yeux = getAttribute("Augen",page)
            plumes = getAttribute("Federn",page)
            peau = getAttribute("Haut",page)
            statut = getAttribute("Status",page)
            spezies = getAttribute("Spezies",page)
            statut = statut if (not statut=="?") else spezies
            eigenschaft = getAttribute("Eigenschaft",page)
            localisation = getAttribute("Verbreitung",page)
        print(f"Insert INTO créatures VALUES('{nom}','{statut}','{yeux}','{peau}','{plumes}','{eigenschaft}','{localisation}')")
        try:
            cursor.execute(f"Insert INTO créatures VALUES('{nom}','{statut}','{yeux}','{peau}','{plumes}','{eigenschaft}','{localisation}')")
        except:
            print(f"Already inserted {nom}")

Insert INTO créatures VALUES('Nagini','Mensch mit (Maledictus-Fluch; früher)','Braun','Blass (Mensch),','?','?','?')
Already inserted Nagini
Insert INTO créatures VALUES('Sir Nicholas de Mimsy-Porpington','Mensch (früher), Geist','Braun','Blass','?','?','?')
Already inserted Sir Nicholas de Mimsy-Porpington
Insert INTO créatures VALUES('Helena Ravenclaw','?','Blau','?','?','?','?')
Already inserted Helena Ravenclaw
Insert INTO créatures VALUES('Hippogreif','?','Orange','?',' Weiß bis Grau','Sehr stolz, Versteht Gesten','Europa')
Already inserted Hippogreif
Insert INTO créatures VALUES('Phönix','?','Braun','?','Scharlachrot, Goldener Schwanz','?','Ägypten, Indien, China')
Already inserted Phönix
Insert INTO créatures VALUES('Peeves','Poltergeist','Schwarz ','Blassweiß','?','?','?')
Already inserted Peeves
Insert INTO créatures VALUES('Fawkes','Phönix','schwarz','pink','Rot und Gold','?','?')
Already inserted Fawkes
Insert INTO créatures VALUES('Dementor','?','Blau','Grau (verschorft)','

Insert INTO créatures VALUES('Bezoar','?','?','?','?','?','?')
Already inserted Bezoar
Insert INTO créatures VALUES('Patrick Delaney-Podmore','Geist','Grau','?','?','?','?')
Already inserted Patrick Delaney-Podmore
Insert INTO créatures VALUES('Geist','?','Gräulich-silbern','Gräulich-silbern','?','Seelen von Tod|verstorbenen Zauberschaft|Zauberern und Hexen, Transparent, Eingeschränkter physischer Einfluss','?')
Already inserted Geist
Insert INTO créatures VALUES('Vlad Drakul','Vampir','Rot','?','?','?','?')
Already inserted Vlad Drakul
Insert INTO créatures VALUES('Minimuff','?','Braun','?','?','?','England, gezüchtet von Fred Weasley|Fred und George Weasley')
Already inserted Minimuff
Insert INTO créatures VALUES('Schlange','?','Unterschiedlich','Unterschiedlich','?','Keine Beine','Verschiedene Gegenden')
Already inserted Schlange
Insert INTO créatures VALUES('Knallrümpfiger Kröter','?','Keine','Bleich','?','?','Ländereien von Hogwarts')
Already inserted Knallrümpfiger Kröter
Insert 

Insert INTO créatures VALUES('Horklumpfeld','?','?','?','?','?','?')
Already inserted Horklumpfeld
Insert INTO créatures VALUES('Wichtelbrunnen','?','?','?','?','?','?')
Already inserted Wichtelbrunnen
Insert INTO créatures VALUES('Poltergeist','?','Unterschiedlich','Unterschiedlich','?','Verursacht Chaos, Ähnlich wie Geister, aber festere Körper, war nie ein lebender Mensch','Weltweit')
Already inserted Poltergeist
Insert INTO créatures VALUES('Vivarium','?','?','?','?','?','?')
Already inserted Vivarium


## Familien

In [9]:
cursor.execute("CREATE TABLE verwandte (erste_name TEXT NOT NULL, zweite_name TEXT NOT NULL, verwandtschaft TEXT, FOREIGN KEY (erste_name) REFERENCES charaktere(name), FOREIGN KEY (zweite_name) REFERENCES charaktere(name))")

In [19]:
def insertRelative(first,second,relation):
    #print(f"Insert INTO verwandte VALUES('{first}','{second}','{relation}')")
    first = first.replace("'","('')")
    second = second.replace("'","('')")
    relation = relation.replace("'","('')")
    try:
        #print(f"Insert INTO verwandte VALUES('{first}','{second}','{relation}')")
        cursor.execute(f"Insert INTO verwandte VALUES('{first}','{second}','{relation}')")
    except:
        print(f"Inserting {first} {second} {relation} didn't work")
    return

pattern = re.compile(r'Familie\s*=\s*(.*?)\s\|', re.S)
kategorien =["[[Kategorie:Weibliche Charaktere]]", "Kategorie:Hexen / Zauberer", "Kategorie:Männliche Charaktere"]
for page in pages:
    if any(kategorie in page.text for kategorie in kategorien) and ("Kategorie:" not in page.title.text):
        nom = getAttribute("Name",page)
        if re.search('Familie\s*=\s*\n\|',page.text): #Related to no-one
            continue
        singleRelative = re.search('Familie\s*=\s*\[\[(.*?)\]\]',page.text)
        if singleRelative:#Related to one person
            onlyRelative = singleRelative.group(1)
            relation = re.search('Familie=.*?\((.*?)\)',page.text)
            if relation:
                insertRelative(onlyRelative,nom,relation.group(1))
            else:
                insertRelative(onlyRelative,nom,"?")
            continue
        multipleRelatives = pattern.search(page.text)
        if multipleRelatives:
            famille_text = multipleRelatives.group(1)
            famille = re.findall(r'\*\[\[([^\]]+)\]\]',famille_text)
            for member in famille:
                search_member = member.replace('|','\|')
                relation = re.search(f'\*\[\[{search_member}\]\].*?\((.*?)\)',page.text)
                if relation:
                    insertRelative(member,nom, relation.group(1))
                else:
                    insertRelative(member,nom, "?")
            
        

In [20]:
connection.commit()
cursor.close()
connection.close()